In [4]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [5]:
import numpy as np
from numba import jit
import numba as nb
import pandas as pd


@jit(nopython=True)
def ewma(X, alpha, adjust=False, ignore_na=False):

    old_wt_factor = 1. - alpha
    new_wt = 1. if adjust else alpha

    n = X.shape[0]
    output = np.empty(n)

    weighted_avg = X[0]
    is_observation = (weighted_avg == weighted_avg)
    nobs = int(is_observation)
    output[0] = weighted_avg if (nobs >= 1) else np.nan
    old_wt = 1.

    for i in range(1, n):
        cur = X[i]
        is_observation = (cur == cur)
        nobs += int(is_observation)
        if weighted_avg == weighted_avg:

            if is_observation or (not ignore_na):

                old_wt *= old_wt_factor
                if is_observation:

                    if weighted_avg != cur:
                        weighted_avg = ((old_wt * weighted_avg) +
                                        (new_wt * cur)) / (old_wt + new_wt)
                    if adjust:
                        old_wt += new_wt
                    else:
                        old_wt = 1.
        elif is_observation:
            weighted_avg = cur

        output[i] = weighted_avg if (nobs >= 1) else np.nan

    return output


@jit(nopython=True)
def ewma_2(X, alpha, adjust=False, ignore_na=False):
    old_wt_factor = 1. - alpha
    new_wt = 1. if adjust else alpha

    n = X.shape[0]
    output = np.empty(n)

    weighted_avg = X[0]
    is_observation = not np.isnan(weighted_avg)
    nobs = int(is_observation)
    output[0] = weighted_avg if is_observation else np.nan
    old_wt = 1.

    for i in range(1, n):
        cur = X[i]
        is_observation = not np.isnan(cur)
        nobs += int(is_observation)

        if is_observation or not ignore_na:
            old_wt *= old_wt_factor
            if is_observation:
                if weighted_avg != cur:
                    weighted_avg = ((old_wt * weighted_avg) + (new_wt * cur)) / (old_wt + new_wt)
                if adjust:
                    old_wt += new_wt
                else:
                    old_wt = 1.
        elif is_observation:
            weighted_avg = cur

        output[i] = weighted_avg if nobs >= 1 else np.nan

    return output


In [6]:
# Example usage
X = np.array([1, 2, 3, 4, 5, 6, 7, 8])
series = pd.Series(X)
alpha = 0.5  # The alpha value for the ewma

smoothed_X = ewma(X, alpha, adjust=True)
print(smoothed_X)
assert np.allclose(smoothed_X, series.ewm(alpha=alpha, adjust=True).mean())


[1.         1.66666667 2.42857143 3.26666667 4.16129032 5.0952381
 6.05511811 7.03137255]


In [7]:
smoothed_X = ewma_2(X, alpha, adjust=True)
print(smoothed_X)
assert np.allclose(smoothed_X, series.ewm(alpha=alpha, adjust=True).mean())

[1.         1.66666667 2.42857143 3.26666667 4.16129032 5.0952381
 6.05511811 7.03137255]


In [8]:
%%timeit
smoothed_X = ewma(X, alpha)

The slowest run took 14.08 times longer than the fastest. This could mean that an intermediate result is being cached.
4 μs ± 5.84 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
smoothed_X = ewma_2(X, alpha)

The slowest run took 12.75 times longer than the fastest. This could mean that an intermediate result is being cached.
4.51 μs ± 6.49 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
